In [1]:
import csv
import requests
import json

In [2]:
# Function to search for a name on LinkedIn
def search_on_linkedin(name):
    url = "https://api.linkedin.com/v2/search"
    headers = {
        "Authorization": "Bearer YOUR_ACCESS_TOKEN",
        "X-RestLi-Protocol-Version": "2.0.0"
    }
    params = { "q": f"firstName:{name.split()[0]} lastName:{name.split()[1]}", "origin": "https://www.linkedin.com/search/results/people/", "count": 10 }

    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:  # Check if the request was successful
        return response.json()
    else:
        print(f"Failed to fetch data: {response.status_code}")
        return {}  # Return an empty dictionary on failure

In [3]:
# Function to extract the top 10 search results and add more attributes
def extract_top_10_results(data):
    results = []
    # Check if 'elements' key exists in the data
    if 'elements' in data:
        for result in data["elements"]:
            # Safely get data with get() to avoid KeyError
            firstName = result.get("firstName", {}).get("localized", {}).get("en_US", "")
            lastName = result.get("lastName", {}).get("localized", {}).get("en_US", "")
            name = f"{firstName} {lastName}"
            location = result.get("location", {}).get("name", {}).get("localized", {}).get("en_US", "")
            headline = result.get("headline", {}).get("localized", {}).get("en_US", "")
            industry = result.get("industry", {}).get("name", {}).get("localized", {}).get("en_US") if "industry" in result else "N/A"
            connection = "1st" if result.get("isConnected", False) else "Not Connected"
            results.append((name, location, headline, industry, connection))
    else:
        print("No 'elements' key in the data")
    return results

In [4]:
# Function to save results in a CSV file
def save_to_csv(results, filename):
    with open(filename, "w", newline="", encoding="utf-8") as csvfile:
        fieldnames = ["Name", "Location", "Headline", "Industry", "Connection"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for result in results:
            writer.writerow({"Name": result[0], "Location": result[1], "Headline": result[2], "Industry": result[3], "Connection": result[4]})

In [7]:
first_name = input("Enter the first name: ")
last_name = input("Enter the last name: ")
name = f"{first_name} {last_name}"
data = search_on_linkedin(name)
results = extract_top_10_results(data)
save_to_csv(results, "linkedin_search_results.csv")

Failed to fetch data: 401
No 'elements' key in the data
